# Remote Downloader for HCP Aspera packages

This is a helper tool to download HCP aspera packages on systems which do not have browser or GUI access. You use your local machine (where you do have browser access, and aspera installed) to navigate to db.humanconnectome.org and initiate a transfer. When you do this, Aspera saves the transfer settings (including a token) to a local file. You can immediately cancel the transfer, then run this tool to initiate that same transfer on a remote system using Aspera CLI.

## Instructions:
- Step 1: In another browser window, use your local machine (i.e. where you do have browser access, and aspera installed, and running this notebook) to navigate to db.humanconnectome.org and initiate a transfer.
- Step 2: Cancel that transfer before it begins to actually download the files.
- Step 3: Run the cells in this notebook, which will read the Aspera transfers db (sql) and pull out the info, including download tokens
- Step 4: You can select the transfer you want to initiate, and a Aspera CLI command will be generated that can initiate the download on a remote system where Aspera CLI is installed.


## Dependencies:

Install Aspera Connect (includes Aspera CLI) on linux (for remote system):
```
#install aspera connect (installs to ~/.aspera):
wget https://download.asperasoft.com/download/sw/connect/3.9.9/ibm-aspera-connect-3.9.9.177872-linux-g2.12-64.tar.gz
tar -xvzf ibm-aspera-connect-3.9.9.177872-linux-g2.12-64.tar.gz
./ibm-aspera-connect-3.9.9.177872-linux-g2.12-64.sh
```

Python modules:
- sqlite3
- json
- subprocess
- os


In [ ]:
from sqlite3 import connect
from json import loads
from subprocess import run
from os.path import expanduser,join

home = expanduser("~")


#on machine where connectomedb browser transfer was initiated
conn = connect(join(home,'.aspera/connect/var/asperaconnect.data'))


##on destination system; if mac os X:
#destination = join(home,'incoming_hcp')
#key = join(home,'Applications/Aspera\ Connect.app/Contents/Resources/asperaweb_id_dsa.openssh')
#ascp = join(home,'Applications/Aspera\ Connect.app/Contents/Resources/ascp')


##destination system; if linux:
#remotehome = '/home/akhanf'
#destination = join(remotehome,'incoming_hcp')
#key = join(remotehome,'.aspera/connect/etc/asperaweb_id_dsa.openssh')
#ascp = join(remotehome,'.aspera/cli/bin/ascp')

In [ ]:
#get entries in local aspera transfer database

c = conn.cursor()
c.execute('SELECT data FROM transfer ORDER BY data DESC')
rows = c.fetchall()

transfer_dicts = [loads(*rows[i]) for i in range(len(rows))]

first_file_list = [transfer_dicts[i]['paths'][0]['source'].split('/')[-1]  for i in range(len(rows))]
token_list = [transfer_dicts[i]['token']  for i in range(len(rows))]
first_file_list
nfiles_list = [len(transfer_dicts[i]['paths'])  for i in range(len(rows))]

#make dropdown list options as the first file, and the number of files:
dropdown_options = [ 'first file: {}, total files = {}'.format(first_file_list[i],nfiles_list[i]) for i in range(len(rows))]

# display a widget
import ipywidgets as widgets

w = widgets.Dropdown(
    options=dropdown_options,
    description='Download:',
    disabled=False,
    layout={'width': 'max-content'}
)
display(w)

In [ ]:
# generate command for local download -- testing only..

#on destination system; if mac os X:
destination = join(home,'incoming_hcp')
key = join(home,'Applications/Aspera\ Connect.app/Contents/Resources/asperaweb_id_dsa.openssh')
ascp = join(home,'Applications/Aspera\ Connect.app/Contents/Resources/ascp')

transfer_dict = transfer_dicts[w.index]

path_list = [path['source'] for path in transfer_dict['paths'] ]
path_string = ' '.join(path_list)


cmdline_opts = '-c {encryption_cipher} --mode recv -O {udp_port} -y 1 -t 443 --host={host}'\
               ' --user={host_user} -i {key} -W {token} -P {host_port} {path_string} {destination}'.format(key=key,path_string=path_string,destination=destination,**transfer_dict)


#print the Aspera CLI that can be used on this local system (e.g. on your Mac) - useful for testing.. 
print(f'{ascp} {cmdline_opts}')

In [ ]:
# generate command for linux download:

transfer_dict = transfer_dicts[w.index]

remotehome = '~'
destination = join(remotehome,'incoming_hcp')
key = join(remotehome,'.aspera/connect/etc/asperaweb_id_dsa.openssh')
ascp = join(remotehome,'.aspera/cli/bin/ascp')


path_list = [path['source'] for path in transfer_dict['paths'] ]
path_string = ' '.join(path_list)


cmdline_opts = '-c {encryption_cipher} --mode recv -O {udp_port} -y 1 -t 443 --host={host}'\
               ' --user={host_user} -i {key} -W {token} -P {host_port} --overwrite=never -k 1 --policy=high {path_string} {destination}'.format(key=key,path_string=path_string,destination=destination,**transfer_dict)

#cmd = f'{cmdline_opts} {path_list} {destination}'
print(f'{ascp} {cmdline_opts}')